<a href="https://colab.research.google.com/github/AryanGupta0419/PytorchTutorial/blob/main/DLLAB4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torchvision import datasets, transforms
from torchvision.io import read_image
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
dataset_url = 'https://www.kaggle.com/datasets/yasserh/titanic-dataset'
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: aryangupta0419
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/yasserh/titanic-dataset


100%|██████████| 22.0k/22.0k [00:00<00:00, 18.8MB/s]

In [ ]:
df = pd.read_csv('/content/titanic-dataset/Titanic-Dataset.csv')

df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})


feature = df.columns[2:]
target = df.columns[1]

df.dropna(inplace=True)
df = df.drop(['Name','Ticket','Cabin'], axis = 1)

x= df.drop('Survived', axis = 1)
y=df['Survived']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)


In [ ]:
input_size = X_train.shape[1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class TitanicModel(nn.Module):
    def __init__(self, input_size):
        super(TitanicModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 48)
        self.fc3 = nn.Linear(48,32)
        self.fc4 = nn.Linear(32, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)  # Apply dropout
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

class TitanicModelNoDropout(nn.Module):
    def __init__(self, input_size):
        super(TitanicModelNoDropout, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 48)
        self.fc3 = nn.Linear(48, 32)
        self.fc4 = nn.Linear(32, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
model = TitanicModel(input_size=X_train.shape[1]).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01) #weight decay is how we add L2 regularization


model1 = TitanicModelNoDropout(input_size=X_train.shape[1]).to(device)

optimizer1 = optim.Adam(model1.parameters(), lr=0.001, weight_decay=0.01)

In [ ]:
epoch = 5000

for epochs in range(epoch):
  model.train()
  optimizer.zero_grad()

  output = model(X_train_tensor)
  loss = criterion(output, y_train_tensor)
  loss.backward()
  optimizer.step()

  if((epochs+1)%500 ==0):
    print(f"Epoch: {epochs+1}, Loss: {loss.item()}")



Epoch: 500, Loss: 0.31626737117767334
Epoch: 1000, Loss: 0.23495043814182281
Epoch: 1500, Loss: 0.199764221906662
Epoch: 2000, Loss: 0.1918654441833496
Epoch: 2500, Loss: 0.22140440344810486
Epoch: 3000, Loss: 0.1864721179008484
Epoch: 3500, Loss: 0.2231733798980713
Epoch: 4000, Loss: 0.17551876604557037
Epoch: 4500, Loss: 0.23070918023586273
Epoch: 5000, Loss: 0.19869032502174377


In [ ]:
epoch = 5000

for epochs in range(epoch):
  model1.train()
  optimizer1.zero_grad()

  output = model1(X_train_tensor)
  loss = criterion(output, y_train_tensor)
  loss.backward()
  optimizer1.step()

  if((epochs+1)%500 ==0):
    print(f"Epoch: {epochs+1}, Loss: {loss.item()}")



Epoch: 500, Loss: 0.07667399942874908
Epoch: 1000, Loss: 0.0616593174636364
Epoch: 1500, Loss: 0.05795777216553688
Epoch: 2000, Loss: 0.05700553581118584
Epoch: 2500, Loss: 0.05651780217885971
Epoch: 3000, Loss: 0.0564345121383667
Epoch: 3500, Loss: 0.05640155449509621
Epoch: 4000, Loss: 0.056336864829063416
Epoch: 4500, Loss: 0.05629561096429825
Epoch: 5000, Loss: 0.056249428540468216


In [ ]:
def evaluate_accuracy(model, X_test_tensor, y_test_tensor):
    model.eval()
    with torch.no_grad():
        output = model(X_test_tensor)
        _, predicted = torch.max(output, 1)
        correct = (predicted == y_test_tensor).sum().item()
        accuracy = correct / len(y_test_tensor)
    return accuracy

test_accuracy_model = evaluate_accuracy(model, X_test_tensor, y_test_tensor)
print(f"Test Accuracy with Dropout: {test_accuracy_model * 100:.2f}%")

test_accuracy_model1 = evaluate_accuracy(model1, X_test_tensor, y_test_tensor)
print(f"Test Accuracy without Dropout: {test_accuracy_model1 * 100:.2f}%")

Test Accuracy with Dropout: 70.27%
Test Accuracy without Dropout: 64.86%
